In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.10.2 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2022 by The Oscar Development Team


In [3]:
#define functions to solve f^* = min f(x) s.t. x in S(g)
#input: f: objective polynomials 
#       g: inequality constraints
#       h:= (g_j-y_j^2)
#       R: ring of poynomials
#       n: number of variables
#       m: number of inequality constraints
#       x: vector of variables
#       lamb: vector of multipliers
#       t: scalar variable
#       radical_comp: compute radical

#output: system of univariate polynomials whose common set of real roots contains f^*

function solve_FJ(n,m,f,g,h,R,x,lamb,t;radical_comp=true)

@time begin
    
#compute the set h_FJ of polynomials from Fritz John conditions
h_FJ=Vector{fmpq_mpoly}(undef,n)
for i in 1:n
    h_FJ[i]=lamb[1]*derivative(f,i)
end
for j in 1:m
    for i in 1:n
        h_FJ[i]-=lamb[j+1]*derivative(g[j], i)
    end
end

for j in 1:m
    h_FJ=[h_FJ;lamb[j+1]*g[j]]
end

h_FJ=[h_FJ;1-sum(lamb.^2)]
h=[h;h_FJ]

h=[h;t[1]-f]#h=(gj(x)-yj^2,h_FJ,x_{n+1}-f)

if radical_comp
    RI = radical(ideal(R, h))#compute radical of V(h)
else
    RI = ideal(R, h)
end
#gens(RI)
    
G=groebner_basis(RI, ordering=lex(gens(R)))
    
check=1
alpha=zeros(Int64,1)
h_uni=Vector{fmpq_mpoly}([])
for j=1:length(G)
    check=1
    for i in 1:length(G[j])
        alpha=exponent_vector(G[j],i) 
        if sum(alpha[t] for t in setdiff(1:n+2*m+2,n+2*m+2))>0
            check=0
        end
    end
    if check==1
        h_uni=[h_uni;G[j]]
    end
end

println(h_uni)# generators of the image f(S(g)\cap V(h_FJ))
    
end
    
end

solve_FJ (generic function with 1 method)

In [4]:
function solve_FJ_plus(n,m,f,g,h,R,x,lamb,t)

@time begin
    
#compute the set h_FJ of polynomials from Fritz John conditions
h_FJ=Vector{fmpq_mpoly}(undef,n)
for i in 1:n
    h_FJ[i]=lamb[1]^2*derivative(f,i)
end
for j in 1:m
    for i in 1:n
        h_FJ[i]-=lamb[j+1]^2*derivative(g[j], i)
    end
end

for j in 1:m
    h_FJ=[h_FJ;lamb[j+1]^2*g[j]]
end

h_FJ=[h_FJ;1-sum(lamb.^2)]
h=[h;h_FJ]

h=[h;t[1]-f]#h=(gj(x)-yj^2,h_FJ,x_{n+1}-f)
    
RI = radical(ideal(R, h))#compute radical of V(h)
#RI = ideal(R, h)
#gens(RI)
    
G=groebner_basis(RI, ordering=lex(gens(R)))
    
check=1
alpha=zeros(Int64,1)
h_uni=Vector{fmpq_mpoly}([])
for j=1:length(G)
    check=1
    for i in 1:length(G[j])
        alpha=exponent_vector(G[j],i) 
        if sum(alpha[t] for t in setdiff(1:n+2*m+2,n+2*m+2))>0
            check=0
        end
    end
    if check==1
        h_uni=[h_uni;G[j]]
    end
end

println(h_uni)# generators of the image f(S(g)\cap V(h_FJ))
    
end
    
end

solve_FJ_plus (generic function with 1 method)

In [5]:
function solve_KKT(n,m,f,g,h,R,x,lamb,t)

@time begin
    
#compute the set h_KKT of polynomials from Karush–Kuhn–Tucker conditions
h_KKT=Vector{fmpq_mpoly}(undef,n)
for i in 1:n
    h_KKT[i]=derivative(f,i)
end
for j in 1:m
    for i in 1:n
        h_KKT[i]-=lamb[j]*derivative(g[j], i)
    end
end

for j in 1:m
    h_KKT=[h_KKT;lamb[j]*g[j]]
end

h=[h;h_KKT]

h=[h;t[1]-f]#h=(gj(x)-yj^2,h_FJ,x_{n+1}-f)
    
RI = radical(ideal(R, h))#compute radical of V(h)
#RI = ideal(R, h)
#gens(RI)
    
G=groebner_basis(RI, ordering=lex(gens(R)))
    
#println(G)
    
check=1
alpha=zeros(Int64,1)
h_uni=Vector{fmpq_mpoly}([])
for j=1:length(G)
    check=1
    for i in 1:length(G[j])
        alpha=exponent_vector(G[j],i) 
        if sum(alpha[t] for t in setdiff(1:n+2*m+1,n+2*m+1))>0
            check=0
        end
    end
    if check==1
        h_uni=[h_uni;G[j]]
    end
end

h_uni# generators of the image f(S(g)\cap V(h_FJ))
    
end
    
end

solve_KKT (generic function with 1 method)

In [6]:
function solve_KKT_plus(n,m,f,g,h,R,x,lamb,t)

@time begin
    
#compute the set h_KKT of polynomials from Karush–Kuhn–Tucker conditions
h_KKT=Vector{fmpq_mpoly}(undef,n)
for i in 1:n
    h_KKT[i]=derivative(f,i)
end
for j in 1:m
    for i in 1:n
        h_KKT[i]-=lamb[j]^2*derivative(g[j], i)
    end
end

for j in 1:m
    h_KKT=[h_KKT;lamb[j]^2*g[j]]
end

h=[h;h_KKT]

h=[h;t[1]-f]#h=(gj(x)-yj^2,h_FJ,x_{n+1}-f)
    
RI = radical(ideal(R, h))#compute radical of V(h)
#RI = ideal(R, h)
#gens(RI)
    
G=groebner_basis(RI, ordering=lex(gens(R)))
    
#println(G)
    
check=1
alpha=zeros(Int64,1)
h_uni=Vector{fmpq_mpoly}([])
for j=1:length(G)
    check=1
    for i in 1:length(G[j])
        alpha=exponent_vector(G[j],i) 
        if sum(alpha[t] for t in setdiff(1:n+2*m+1,n+2*m+1))>0
            check=0
        end
    end
    if check==1
        h_uni=[h_uni;G[j]]
    end
end

h_uni# generators of the image f(S(g)\cap V(h_FJ))
    
end
    
end

solve_KKT_plus (generic function with 1 method)

In [7]:
function proj_coordinate(n,m,f,g,h,R,x,t,r;radical_comp=true)

@time begin
    
#compute the set h_FJ of polynomials from Fritz John conditions
h_FJ=Vector{fmpq_mpoly}(undef,n)
for i in 1:n
    h_FJ[i]=lamb[1]*derivative(f,i)
end
for j in 1:m
    for i in 1:n
        h_FJ[i]-=lamb[j+1]*derivative(g[j], i)
    end
end

for j in 1:m
    h_FJ=[h_FJ;lamb[j+1]*g[j]]
end

h_FJ=[h_FJ;1-sum(lamb.^2)]
h=[h;h_FJ]

h=[h;t[1]-x[r];f]#h=(gj(x)-yj^2,h_FJ,x_{n+1}-f)

if radical_comp
    RI = radical(ideal(R, h))#compute radical of V(h)
else
    RI = ideal(R, h)
end
#gens(RI)
    
G=groebner_basis(RI, ordering=lex(gens(R)))
    
check=1
alpha=zeros(Int64,1)
h_uni=Vector{fmpq_mpoly}([])
for j=1:length(G)
    check=1
    for i in 1:length(G[j])
        alpha=exponent_vector(G[j],i) 
        if sum(alpha[t] for t in setdiff(1:n+2*m+2,n+2*m+2))>0
            check=0
        end
    end
    if check==1
        h_uni=[h_uni;G[j]]
    end
end

println(h_uni)# generators of the image f(S(g)\cap V(h_FJ))
    
end
    
end

proj_coordinate (generic function with 1 method)

In [7]:
#Example 1 in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=2
m=2

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))

f=x[2]#objective polynomial
g=[-x[1];x[1]-x[2]^2]#inequality constraints


h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_FJ(n,m,f,g,h,R,x,lamb,t)

fmpq_mpoly[t_{1}]
  3.412440 seconds (2.61 M allocations: 140.025 MiB, 98.98% compilation time)


In [8]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = u

r = roots(m)

1-element Vector{acb}:
 0

In [9]:
#Example 2 in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=2
m=3

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m), "t" => (1:1))

f=x[1]-5*x[2]#objective polynomial

g=[x[1]^2-x[2];-x[1]^2+4*x[2];-x[2]+1]#inequality constraints


h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_KKT(n,m,f,g,h,R,x,lamb,t)

  0.017167 seconds (43.94 k allocations: 1.026 MiB)


1-element Vector{fmpq_mpoly}:
 100*t_{1}^6 + 1975*t_{1}^5 + 14001*t_{1}^4 + 41395*t_{1}^3 + 39295*t_{1}^2 - 12150*t_{1} + 504

In [10]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = 100*u^6 + 1975*u^5 + 14001*u^4 + 41395*u^3 + 39295*u^2 - 12150*u + 504

r = roots(m,isolate_real = true)

6-element Vector{acb}:
 [-7.00000 +/- 3.20e-6]
 [-6.00000 +/- 4.11e-6]
 [-4.00000 +/- 1.24e-6]
 [-3.000000 +/- 5.76e-7]
 [0.050000000 +/- 4.06e-10]
 [0.200000000 +/- 9.30e-10]

In [11]:
#[11, Example 3] in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=2
m=1

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))

f=(x[1]+1)^2+x[2]^2-1

g=[x[1]^3-x[2]^2]


h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_FJ(n,m,f,g,h,R,x,lamb,t,radical_comp=true)

fmpq_mpoly[27*t_{1}^4 + 59*t_{1}^3 + 60*t_{1}^2 + 28*t_{1}]
  0.014648 seconds (12.04 k allocations: 301.051 KiB, 44.32% compilation time)


In [12]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = 27*u^4 + 59*u^3 + 60*u^2 + 28*u

r = roots(m)

4-element Vector{acb}:
 [-1.0000000 +/- 5.42e-9]
 [+/- 3.16e-29]
 [-0.5925926 +/- 1.65e-8] + [0.8281733 +/- 3.42e-8]*im
 [-0.5925926 +/- 1.46e-8] + [-0.8281733 +/- 3.26e-8]*im

In [13]:
#The case t=-1
#Finding x_1^*

using Oscar

n=2
m=1

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))

f=(x[1]+1)^2+x[2]^2

g=[x[1]^3-x[2]^2]


h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

proj_coordinate(n,m,f,g,h,R,x,t,1)

fmpq_mpoly[t_{1} + 1]
  0.169953 seconds (148.73 k allocations: 8.110 MiB, 97.01% compilation time)


In [14]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = u + 1

r = roots(m)

1-element Vector{acb}:
 -1.0000000000000000000

In [15]:
#The case t=-1
#Finding x_2^*

using Oscar

n=2
m=1

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))

f=(x[1]+1)^2+x[2]^2

g=[x[1]^3-x[2]^2]


h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

proj_coordinate(n,m,f,g,h,R,x,t,2)

fmpq_mpoly[t_{1}]
  0.005697 seconds (1.66 k allocations: 56.930 KiB)


In [16]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = u

r = roots(m)

1-element Vector{acb}:
 0

In [17]:
#The case t=0
#Finding x_1^*

using Oscar

n=2
m=1

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))

f=(x[1]+1)^2+x[2]^2-1

g=[x[1]^3-x[2]^2]


h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

proj_coordinate(n,m,f,g,h,R,x,t,1)

fmpq_mpoly[t_{1}]
  0.005203 seconds (1.55 k allocations: 53.898 KiB)


In [18]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = u

r = roots(m)

1-element Vector{acb}:
 0

In [19]:
#The case t=0
#Finding x_2^*

using Oscar

n=2
m=1

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))

f=(x[1]+1)^2+x[2]^2-1

g=[x[1]^3-x[2]^2]


h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

proj_coordinate(n,m,f,g,h,R,x,t,2)

fmpq_mpoly[t_{1}]
  0.004928 seconds (1.55 k allocations: 53.898 KiB)


In [20]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = u

r = roots(m)

1-element Vector{acb}:
 0

In [21]:
#[11, Example 5] in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=2
m=1

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))

f=x[1]-x[2]

g=[(x[1]-x[2])^3]


h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_FJ_plus(n,m,f,g,h,R,x,lamb,t)

fmpq_mpoly[t_{1}]
  0.041001 seconds (2.40 k allocations: 90.219 KiB)


In [22]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = u

r = roots(m)

1-element Vector{acb}:
 0

In [23]:
#[6, Example A2] in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=2
m=1

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m), "t" => (1:1))

f=(x[1]^2+x[2]^2-2)*(x[1]^2+x[2]^2)#objective polynomial

g=[(x[1]^2+x[2]^2-2)*(x[1]-3)]#inequality constraints

h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_KKT(n,m,f,g,h,R,x,lamb,t)

  0.059155 seconds (17.23 k allocations: 783.531 KiB)


1-element Vector{fmpq_mpoly}:
 t_{1}^3 - 62*t_{1}^2 - 63*t_{1}

In [24]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = u^3 - 62*u^2 - 63*u

r = roots(m,isolate_real = true)

3-element Vector{acb}:
 [-1.00000000 +/- 1.40e-9]
 [+/- 6.08e-29]
 [63.00000000 +/- 1.40e-9]

In [25]:
#[11, Example 11] in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=3
m=1

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m), "t" => (1:1))

eps=1//10

f=x[1]^4*x[2]^2+x[1]^2*x[2]^4+x[3]^6-3*x[1]^2*x[2]^2*x[3]^2+eps*(x[1]^2+x[2]^2+x[3]^2) 
g=[1-x[1]^2-x[2]^2-x[3]^2]

h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_KKT(n,m,f,g,h,R,x,lamb,t)

  0.157770 seconds (1.44 M allocations: 31.932 MiB)


1-element Vector{fmpq_mpoly}:
 49207500000000000*t_{1}^11 - 81961242187500000*t_{1}^10 + 35401891640625000*t_{1}^9 - 5747811890625000*t_{1}^8 + 576144305156250*t_{1}^7 - 50773802062500*t_{1}^6 + 2487391238125*t_{1}^5 - 34983679000*t_{1}^4 + 1586740350*t_{1}^3 - 4087040*t_{1}^2 + 182336*t_{1}

In [26]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = 49207500000000000*u^11 - 81961242187500000*u^10 + 35401891640625000*u^9 - 5747811890625000*u^8 + 576144305156250*u^7 - 50773802062500*u^6 + 2487391238125*u^5 - 34983679000*u^4 + 1586740350*u^3 - 4087040*u^2 + 182336*u

r = roots(m,isolate_real = true)

11-element Vector{acb}:
 [+/- 1.22e-11]
 [0.100000 +/- 2.57e-8]
 [0.1156250 +/- 3.35e-8]
 [0.3500000 +/- 2.03e-8]
 [1.1000000 +/- 1.89e-8]
 [+/- 1.82e-10] + [0.012171612 +/- 5.68e-10]*im
 [+/- 1.22e-10] + [-0.012171612 +/- 5.12e-10]*im
 [+/- 3.29e-10] + [0.024343225 +/- 5.47e-10]*im
 [+/- 3.29e-10] + [-0.024343225 +/- 5.47e-10]*im
 [+/- 1.32e-9] + [0.09737290 +/- 2.18e-9]*im
 [+/- 1.35e-9] + [-0.09737290 +/- 2.21e-9]*im

In [27]:
#[11, Example 12] in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=2
m=3

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m), "t" => (1:1))


f=x[1]*x[2]+x[1]^3+x[2]^3
g=[x[1];x[2];1-x[1]-x[2]]

h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_KKT(n,m,f,g,h,R,x,lamb,t)

  0.013638 seconds (20.35 k allocations: 447.516 KiB)


1-element Vector{fmpq_mpoly}:
 54*t_{1}^4 - 83*t_{1}^3 + 30*t_{1}^2 - t_{1}

In [28]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = 54*u^4 - 83*u^3 + 30*u^2 -u

r = roots(m,isolate_real = true)

4-element Vector{acb}:
 0
 [0.037037037 +/- 1.73e-10]
 [0.50000000 +/- 3.52e-9]
 1.0000000000000000000

In [29]:
#[11, Example 13] in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=3
m=3

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m), "t" => (1:1))


f=x[1]^4*x[2]^2+x[2]^4*x[3]^2+x[3]^4*x[1]^2-3*x[1]^2*x[2]^2*x[3]^2
g=[1-x[1]^2;1-x[2]^2;1-x[3]^2]

h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_KKT_plus(n,m,f,g,h,R,x,lamb,t)

  1.143862 seconds (724.93 k allocations: 14.630 MiB, 0.44% compilation time)


1-element Vector{fmpq_mpoly}:
 64*t_{1}^4 - 271*t_{1}^3 + 234*t_{1}^2 - 27*t_{1}

In [30]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = 64*u^4 - 271*u^3 + 234*u^2 - 27*u

r = roots(m,isolate_real = true)

4-element Vector{acb}:
 [+/- 6.43e-26]
 [0.13616744 +/- 3.33e-9]
 [1.0000000 +/- 6.40e-9]
 [3.09820756 +/- 7.77e-9]

In [40]:
#[12, Example 5] in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=2
m=3

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))


f=x[1]+x[2]
g=[x[1];x[2];-x[1]*x[2]]

h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_KKT_plus(n,m,f,g,h,R,x,lamb,t)

  0.093578 seconds (2.72 k allocations: 95.016 KiB)


1-element Vector{fmpq_mpoly}:
 t_{1}

In [52]:
#Example 3 in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=2
m=2*n+1

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))

f=sum(x)
M=[-2  -1;
 -1  -4]
q=-M*ones(Int64,n)
g=[x;M*(x.^2)+q;-sum(x[j]^2*(sum(M[j,r]*x[r]^2 for r=1:n)+q[j]) for j=1:n)]

h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_FJ(n,m,f,g,h,R,x,lamb,t)

fmpq_mpoly[12544*t_{1}^25 - 404544*t_{1}^23 + 5722416*t_{1}^21 - 46888716*t_{1}^19 + 248061620*t_{1}^17 - 896757033*t_{1}^15 + 2295187042*t_{1}^13 - 4262362971*t_{1}^11 + 5819482184*t_{1}^9 - 5801398680*t_{1}^7 + 4059214650*t_{1}^5 - 1806273000*t_{1}^3 + 388800000*t_{1}]
 96.969092 seconds (63.29 M allocations: 109.773 GiB, 5.93% gc time)


In [49]:
q

2-element Vector{Int64}:
 3
 5

In [53]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

unipol = 12544*u^25 - 404544*u^23 + 5722416*u^21 - 46888716*u^19 + 248061620*u^17 - 896757033*u^15 + 2295187042*u^13 - 4262362971*u^11 + 5819482184*u^9 - 5801398680*u^7 + 4059214650*u^5 - 1806273000*u^3 + 388800000*u

r = roots(unipol,isolate_real = true)

25-element Vector{acb}:
 [-2.500000000000 +/- 1.25e-13]
 [-2.23606797750 +/- 3.05e-12]
 [-2.1213203436 +/- 5.16e-11]
 [-2.0225705874 +/- 7.44e-11]
 [-2.0000000000 +/- 2.88e-11]
 [-1.732050807569 +/- 4.68e-13]
 [-1.2247448713916 +/- 5.06e-14]
 [-1.1180339887499 +/- 2.19e-14]
 [+/- 1.24e-62]
 [1.1180339887499 +/- 2.24e-14]
 [1.2247448713916 +/- 4.09e-14]
 [1.732050807569 +/- 4.62e-13]
 [2.0000000000 +/- 2.76e-11]
 [2.0225705874 +/- 8.12e-11]
 [2.1213203436 +/- 5.23e-11]
 [2.23606797750 +/- 2.68e-12]
 [2.500000000000 +/- 1.08e-13]
 [-1.0282232959291 +/- 2.43e-14] + [0.3780917977261 +/- 1.76e-14]*im
 [-1.0282232959291 +/- 2.60e-14] + [-0.3780917977261 +/- 1.93e-14]*im
 [1.0282232959291 +/- 2.35e-14] + [0.3780917977261 +/- 1.67e-14]*im
 [1.0282232959291 +/- 2.35e-14] + [-0.3780917977261 +/- 1.67e-14]*im
 [-0.92321492049484 +/- 8.33e-15] + [-0.66106839191873 +/- 9.09e-15]*im
 [-0.92321492049484 +/- 8.83e-15] + [0.66106839191873 +/- 9.58e-15]*im
 [0.92321492049484 +/- 7.72e-15] + [0.661068391

In [54]:
#The case t=0
#Finding x_1^*

using Oscar

n=2
m=2*n+1

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))

f=sum(x)

g=[x;M*(x.^2)+q;-sum(x[j]^2*(sum(M[j,r]*x[r]^2 for r=1:n)+q[j]) for j=1:n)]


h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

proj_coordinate(n,m,f,g,h,R,x,t,1)

fmpq_mpoly[t_{1}^3 - t_{1}]
  0.180915 seconds (41.30 k allocations: 2.437 MiB)


In [56]:
#The case t=0
#Finding x_2^*

using Oscar

n=2
m=2*n+1

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))

f=sum(x)

g=[x;M*(x.^2)+q;-sum(x[j]^2*(sum(M[j,r]*x[r]^2 for r=1:n)+q[j]) for j=1:n)]


h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

proj_coordinate(n,m,f,g,h,R,x,t,2)

fmpq_mpoly[t_{1}^3 - t_{1}]
  0.180624 seconds (41.30 k allocations: 2.436 MiB)
